## Transfer Train a ResNet Classifier
Potential improvement: bigger batch size

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from load_dataset import process_img, load_dataset
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [2]:
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# https://towardsdatascience.com/how-to-train-an-image-classifier-in-pytorch-and-use-it-to-perform-basic-inference-on-single-images-99465a1e9bf5
os.chdir('/Users/joeholt/Documents/College Local/Current/CS 639/proj/test') 
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data/gesture_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [4]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names))
model_ft.fc = nn.Linear(num_ftrs, 8)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [5]:
checkpoint = torch.load('checkpoint.pth')
model_ft.load_state_dict(checkpoint['model'])
optimizer_ft.load_state_dict(checkpoint['optimizer'])
exp_lr_scheduler.load_state_dict(checkpoint['lr_sched'])

In [6]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=15)

Epoch 0/14
----------
train Loss: 0.6729 Acc: 0.7505
val Loss: 0.0016 Acc: 1.0000

Epoch 1/14
----------
train Loss: 0.5969 Acc: 0.7766
val Loss: 0.0069 Acc: 1.0000

Epoch 2/14
----------
train Loss: 0.5699 Acc: 0.7873
val Loss: 0.0108 Acc: 1.0000

Epoch 3/14
----------
train Loss: 0.5467 Acc: 0.7953
val Loss: 0.0044 Acc: 1.0000

Epoch 4/14
----------
train Loss: 0.5626 Acc: 0.7844
val Loss: 0.0055 Acc: 1.0000

Epoch 5/14
----------
train Loss: 0.5416 Acc: 0.7953
val Loss: 0.0159 Acc: 1.0000

Epoch 6/14
----------
train Loss: 0.5530 Acc: 0.7905
val Loss: 0.0059 Acc: 1.0000

Epoch 7/14
----------
train Loss: 0.5425 Acc: 0.7927
val Loss: 0.0035 Acc: 1.0000

Epoch 8/14
----------
train Loss: 0.5406 Acc: 0.7990
val Loss: 0.0075 Acc: 1.0000

Epoch 9/14
----------
train Loss: 0.5343 Acc: 0.7964
val Loss: 0.0109 Acc: 1.0000

Epoch 10/14
----------
train Loss: 0.5254 Acc: 0.8003
val Loss: 0.0075 Acc: 1.0000

Epoch 11/14
----------
train Loss: 0.5506 Acc: 0.7901
val Loss: 0.0092 Acc: 1.0000

Ep

In [7]:
torch.save(model_ft, 'resenet-v0-25ep.pth')

In [8]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=2)

Epoch 0/1
----------
train Loss: 0.6646 Acc: 0.7584
val Loss: 0.0028 Acc: 1.0000

Epoch 1/1
----------
train Loss: 0.6623 Acc: 0.7548
val Loss: 0.0030 Acc: 1.0000

Training complete in 64m 47s
Best val Acc: 1.000000


In [9]:
torch.save(model_ft, 'resenet-v0-27ep.pth')

In [11]:
checkpoint = { 
    'epoch': 6,
    'model': model_ft.state_dict(),
    'optimizer': optimizer_ft.state_dict(),
    'lr_sched': exp_lr_scheduler.state_dict()
}
torch.save(checkpoint, 'checkpoint.pth')